这是一个 minimal nnvm example.
整个 workflow 就是先在第二个 code block 定义出合适的 computation graph, 然后在第三个 code block 进行反向。

由于反向节点的 `_conv2d_grad` 等节点没有 `FTVMCompute` 和 `FTVMSchedule` ，所以目前会 block

In [1]:
import nnvm
import tvm
from nnvm import symbol
from nnvm import graph

In [2]:
x = symbol.Variable("x")
conv1 = symbol.conv2d(x, channels=20, kernel_size=[3, 3], name="conv1", use_bias=True)
out_sum = symbol.sum(conv1)

In [4]:
ys = out_sum
xs = x

g = graph.create(out_sum)
g._set_symbol_list_attr('grad_ys', ys)
g._set_symbol_list_attr('grad_xs', xs)
ny = len(ys.list_output_names())
grad_ys = [symbol.ones_like(ys[i]) for i in range(ny)]
g._set_symbol_list_attr('grad_ys_out_grad', grad_ys)
sym = g.apply('Gradient').symbol

nx = len(xs) if isinstance(xs, list) else len(xs.list_output_names())
ret = [sym[i] for i in range(nx)]
print(len(ret))
print("---")
print(graph.create(ret[0]).json())
print("---")
print(graph.create(ret[0]).ir())

1
---
{
  "nodes": [
    {
      "op": "null", 
      "name": "x", 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "conv1_weight", 
      "attrs": {
        "channels": "20", 
        "kernel_size": "[3, 3]", 
        "use_bias": "True"
      }, 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "conv1_bias", 
      "attrs": {
        "channels": "20", 
        "kernel_size": "[3, 3]", 
        "use_bias": "True"
      }, 
      "inputs": []
    }, 
    {
      "op": "conv2d", 
      "name": "conv1", 
      "attrs": {
        "channels": "20", 
        "kernel_size": "[3, 3]", 
        "use_bias": "True"
      }, 
      "inputs": [[0, 0, 0], [1, 0, 0], [2, 0, 0]]
    }, 
    {
      "op": "sum", 
      "name": "sum0", 
      "inputs": [[3, 0, 0]]
    }, 
    {
      "op": "ones_like", 
      "name": "ones_like1", 
      "inputs": [[4, 0, 0]]
    }, 
    {
      "op": "expand_like", 
      "name": "sum0_grad", 
      "attrs": {
        "axis": "[]"

In [5]:
g2 = graph.create(ret[0])
deploy_graph, lib, params = nnvm.compiler.build(g2, target=tvm.target.arm_cpu(), shape={"x": (1,28,28,3)}, dtype="float32")

...100%, 0.77 MB, 646 KB/s, 1 seconds passed


Cannot find config for target=llvm -device=arm_cpu -model=unknown, workload=('conv2d', (1, 28, 28, 3, 'float32'), (20, 28, 3, 3, 'float32'), (1, 1), (0, 0), (1, 1), 'NCHW', 'float32'). A fallback configuration is used, which may bring great performance regression.


NNVMError: [14:42:03] /Users/xuxiaoqiao/git-repos/tvm/nnvm/include/nnvm/op.h:532: Check failed: idx < data_.size() && data_[idx].second: Attribute FTVMCompute has not been registered for Operator _conv2d_grad
Stack trace:
  [bt] (0) 1   libnnvm_compiler.dylib              0x00000001274d5cb3 dmlc::LogMessageFatal::~LogMessageFatal() + 67
  [bt] (1) 2   libnnvm_compiler.dylib              0x00000001274d39c5 dmlc::LogMessageFatal::~LogMessageFatal() + 21
  [bt] (2) 3   libnnvm_compiler.dylib              0x000000012752935b nnvm::OpMap<std::__1::function<tvm::Array<tvm::Tensor, void> (nnvm::NodeAttrs const&, tvm::Array<tvm::Tensor, void> const&, tvm::Array<tvm::Tensor, void> const&)> >::operator[](nnvm::Op const*) const + 603
  [bt] (3) 4   libnnvm_compiler.dylib              0x00000001275262a2 nnvm::compiler::CompileEngine::GetScheduleArgs(nnvm::Graph, tvm::Array<tvm::Tensor, void> const&, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&, int, s

In [ ]:
print(lib.imported_modules[0].get_source())